# Dataset Splitter or Reducer

In [1]:
import pandas as pd
import numpy as np

In [3]:
dataset = pd.read_json('../Datasets/VDISC/train_clang.json',orient='records', lines=True)

In [5]:
dataset=dataset.iloc[:,[0,6]]

,functionSource,combine
0,"[clear_area, (, int, startx, ,, int, starty, ,...",False
1,"[ReconstructDuList, (, Statement, *, head, ), ...",False
2,"[free_speaker, (, void, ), {, if, (, Lengths, ...",False
3,"[mlx4_register_device, (, struct, mlx4_dev, *,...",False
4,"[Parse_Env_Var, (, void, ), {, char, *, p, =, ...",True
...,...,...
1019466,"[visitICmpInst, (, ICmpInst, &, I, ), {, bool,...",False
1019467,"[path_node_delete_chain, (, path_node, *, head...",False
1019468,"[addrconf6_start, (, NMDevice, *, self, ), {, ...",False
1019469,"[Next, (, const, SQObjectPtr, &, refpos, ,, SQ...",False


In [6]:
dataset.iloc[:,1] = np.multiply(dataset.iloc[:,1],1)
dataset = dataset.rename(columns={'functionSource':'codes', 'combine':'label'})
false = dataset[dataset.iloc[:,1]==0]
true = dataset[dataset.iloc[:,1]==1]
train = false[0:20000].append(true[0:1333])
test  = false[20000:22494].append(true[2000:2166])
valid = false[30000:32494].append(true[3000:3166])

## Shuffle
train = train.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)
valid = valid.sample(frac=1).reset_index(drop=True)

## Save to json
train.to_json('../Datasets/VDISC/train_ratio_clang.json', orient='records',lines=True)
test.to_json('../Datasets/VDISC/test_ratio_clang.json', orient='records',lines=True)
valid.to_json('../Datasets/VDISC/valid_ratio_clang.json', orient='records',lines=True)

In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn import model_selection, linear_model, tree, ensemble, svm, neural_network, metrics
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import LabelEncoder

In [19]:
mydata = pd.read_csv('cancer.csv')

In [20]:
mydata = mydata.iloc[:,:31]

In [21]:
mydata = mydata.iloc[:,1:]

In [22]:
label_encoder = LabelEncoder()
mydata.iloc[:,0] = label_encoder.fit_transform(mydata.iloc[:,0]).astype('float64')

In [23]:
x = mydata.iloc[:,1:]
y = mydata.iloc[:,0]

In [24]:
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(x, y, train_size=0.7, test_size=0.3, random_state=1234)

In [77]:
sel = SelectFromModel(tree.DecisionTreeClassifier(criterion='gini', random_state=1234))
sel.fit(X_train, y_train)
sel.get_support()

array([False, False,  True, False, False, False,  True, False, False])

In [81]:
model_D3 = tree.DecisionTreeClassifier(criterion='gini', random_state=1234)
model_D3 = model_D3.fit(X=X_train, y=y_train)
model_D3.score(X=X_train, y=y_train)

1.0

In [78]:
sel.estimator_.feature_importances_

array([0.01216977, 0.00865406, 0.74156005, 0.01040987, 0.04120857,
       0.06037519, 0.11241787, 0.        , 0.01320463])

In [82]:
output = model_D3.predict(X_test)
output_prob = model_D3.predict_proba(X_test)

In [52]:
confusion = metrics.confusion_matrix(y_true=y_test, y_pred=output)
print('Confusion matrix: \n',confusion)

tn, fp, fn, tp = confusion.ravel()
print('\nTP:',tp)
print('FP:',fp)
print('TN:',tn)
print('FN:',fn)

## Performance measure
print('\nAccuracy: '+ str(metrics.accuracy_score(y_true=y_test, y_pred=output)))
print('Precision: '+ str(metrics.precision_score(y_true=y_test, y_pred=output)))
print('Recall: '+ str(metrics.recall_score(y_true=y_test, y_pred=output)))
print('F-measure: '+ str(metrics.f1_score(y_true=y_test, y_pred=output)))
print('Area Under the Curve: '+ str(metrics.roc_auc_score(y_true=y_test, y_score=output_prob[:,1])))

Confusion matrix: 
 [[100   5]
 [ 11  55]]

TP: 55
FP: 5
TN: 100
FN: 11

Accuracy: 0.9064327485380117
Precision: 0.9166666666666666
Recall: 0.8333333333333334
F-measure: 0.8730158730158729
Area Under the Curve: 0.8928571428571429


###tukar

In [79]:
cond = sel.estimator_.feature_importances_ !=0
selected_col = X_train.columns[cond]
X_train = X_train[selected_col]
X_test = X_test[selected_col]

In [83]:
confusion = metrics.confusion_matrix(y_true=y_test, y_pred=output)
print('Confusion matrix: \n',confusion)

tn, fp, fn, tp = confusion.ravel()
print('\nTP:',tp)
print('FP:',fp)
print('TN:',tn)
print('FN:',fn)

## Performance measure
print('\nAccuracy: '+ str(metrics.accuracy_score(y_true=y_test, y_pred=output)))
print('Precision: '+ str(metrics.precision_score(y_true=y_test, y_pred=output)))
print('Recall: '+ str(metrics.recall_score(y_true=y_test, y_pred=output)))
print('F-measure: '+ str(metrics.f1_score(y_true=y_test, y_pred=output)))
print('Area Under the Curve: '+ str(metrics.roc_auc_score(y_true=y_test, y_score=output_prob[:,1])))

Confusion matrix: 
 [[100   5]
 [  8  58]]

TP: 58
FP: 5
TN: 100
FN: 8

Accuracy: 0.9239766081871345
Precision: 0.9206349206349206
Recall: 0.8787878787878788
F-measure: 0.8992248062015504
Area Under the Curve: 0.9155844155844156


In [84]:
X_train.shape

(398, 8)